In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

# 各種数値

In [19]:
grav = 9.81

In [100]:
# 順に、軸荷重、径荷重、軸振動数、径振動数
rocket = np.array([6.0, 2.5, 30.0, 10.0])
reentry = np.array([100.0, 0.0, 0.0, 0.0])

#要求を決定
required = np.max([rocket,reentry],axis=0)

In [93]:
torayca_t800h = {'density':1.6, #g/cm^3
                 't　ension':2850, #MPa
                 'comp':1700, #MPa
                 'E':155, #GPa
                 'poisson':0.3  
                }

kevlar49 = {'density':1.45, #g/cm^3
            't　ension':3000, #MPa
            'comp':300, #MPa
            'E':112.4, #GPa
            'poisson':0.3  
            }

al7075 = {'density':2.8, #g/cm^3
          't　ension':550, #MPa
          'comp':500, #MPa
          'E':70, #GPa
          'shear':330, #MPa
          'poisson':0.3  
         }

In [94]:
def get_k_c(material,L,r,t):
    Z_L = L**2/r/t * np.sqrt(1-material['poisson']**2)
    k_c = (10**(-0.6))*Z_L
    return k_c

In [95]:
def get_sigma_buckling_cylinder(material,L,r,t):
    k_c = get_k_c(material,L,r,t)
    E = material['E'] *1e9
    nu = material['poisson']
    return k_c*np.pi**2*E/12/((1-nu)**2)*(t/L)**2

In [96]:
def get_sigma(required,L,r,t,mass):
    """
    最終的にこのsigmaがsigma_buckling_cylinderを下回っていることを確認したい
    """
    F = required[0] * mass * grav
    A = 2*np.pi*r*t
    return F/A

In [97]:
def get_f_axis(material,M,L,r,t):
    A = 2*np.pi*r*t
    E = material['E'] *1e9
    f_axis = 0.250*np.sqrt(A*E/M/L)
    return f_axis

def get_f_rad(material,M,L,r,t):
    E = material['E']*1e9
    I = np.pi*(L**4-(L-t)**4)/64
    f_rad = 0.560*np.sqrt(E*I/M/L**3)
    return f_rad

In [98]:
def check(material,L,r,t,mass):
    log = ['不合格','合格']
    
    sigma_buckling_cylinder = get_sigma_buckling_cylinder(material,L,r,t)
    sigma = get_sigma(required,L,r,t,mass)
    print('=================================')
    print('縦荷重:{0}'.format(sigma))
    print('縦荷重許容値:{0}'.format(sigma_buckling_cylinder))
    print('縦荷重合否:{0}'.format(log[sigma_buckling_cylinder>sigma]))
    print('縦荷重安全率:{0}\n'.format(sigma_buckling_cylinder/sigma))
    
    print('=================================')    
    f_axis = get_f_axis(material,mass,L,r,t)
    f_axis_req = required[2]
    print('縦振動数:{0}'.format(f_axis))
    print('縦振動数最低値:{0}'.format(f_axis_req))
    print('縦振動数合否:{0}'.format(log[f_axis>f_axis_req]))
    print('縦振動数安全率:{0}\n'.format(f_axis/f_axis_req))
    
    print('=================================')    
    f_rad = get_f_rad(material,mass,L,r,t)
    f_rad_req = required[3]
    print('横振動数:{0}'.format(f_rad))
    print('横振動数最低値:{0}'.format(f_rad_req))
    print('横振動数合否:{0}'.format(log[f_rad>f_rad_req]))
    print('横振動数安全率:{0}\n'.format(f_rad/f_rad_req))

In [101]:
#試しに1段目について
material = al7075
L = 1.5
r = 1.0
t = 0.003
mass = 900 + 700

check(material,L,r,t,mass)

縦荷重:83269866.22567964
縦荷重許容値:84462204.05197558
縦荷重合否:合格
縦荷重安全率:1.014318959310736

縦振動数:185.36766074112919
縦振動数最低値:30.0
縦振動数合否:合格
縦振動数安全率:6.178922024704306

横振動数:89.76376869806792
横振動数最低値:10.0
横振動数合否:合格
横振動数安全率:8.976376869806792



/Users/KojiMinoda/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  if __name__ == '__main__':
/Users/KojiMinoda/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
/Users/KojiMinoda/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


In [91]:
#試しに1段目について
material = kevlar49
L = 0.6
r = 0.5
t = 0.01
mass = 100

check(material,L,r,t,mass)

縦荷重:24980959.867703892
縦荷重許容値:904147784.3278147
縦荷重合否:合格
縦荷重安全率:36.19347651635769

縦振動数:1917.8846155538267
縦振動数最低値:30.0
縦振動数合否:合格
縦振動数安全率:63.929487185127556

横振動数:821.5803324796384
横振動数最低値:10.0
横振動数合否:合格
横振動数安全率:82.15803324796384



/Users/KojiMinoda/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  if __name__ == '__main__':
/Users/KojiMinoda/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
/Users/KojiMinoda/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
